# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [23]:
%load_ext dotenv
%dotenv




The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [24]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [25]:
import os
from glob import glob

# get path to the price data 
price_data_dir = os.getenv("PRICE_DATA")

# access th ecollection of all files
price_files = glob(os.path.join(price_data_dir, "**/*.parquet"), recursive= True)



For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [26]:
# we read parquet file
dd_price = dd.read_parquet(price_files).set_index("Ticker")

# add new column as Close_lag_1 
dd_shift = dd_price.groupby("Ticker", group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x["Close"].shift(1))
)

# add return calculation
dd_returns = dd_shift.assign(
    retuns = lambda x: x["Close"]/x["Close_lag_1"]-1
)

# add Hi and Lo range
dd_feat = dd_returns.assign(
    hi_lo_range = lambda x: x["High"] - x["Low"]
)



/var/folders/_h/6ltmmtqn4m3d309fyknwb98c0000gn/T/ipykernel_94681/2342129137.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_price.groupby("Ticker", group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [27]:

# convert Dask data frame to pands
pd_feat = dd_feat.compute()

# Calculate moving average return
pd_ma_returns = pd_feat.groupby("Ticker", group_keys=False).apply(
    lambda x: x.assign(ma_returns = x["retuns"].rolling(10).mean())
)

pd_ma_returns.head()

Price,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,retuns,hi_lo_range,ma_returns
Ticker,,,,,,,,,,,,
DOV,2010-01-04,21.331858,28.395418,28.395418,27.906073,28.100470,1859815.0,2010,NaN,NaN,0.489346,NaN
DOV,2010-01-05,21.231140,28.261351,28.361900,28.120581,28.328384,1272199.0,2010,28.395418,-0.004721,0.241320,NaN
DOV,2010-01-06,21.482927,28.596518,28.636738,28.087063,28.254646,1852654.0,2010,28.261351,0.011860,0.549675,NaN
DOV,2010-01-07,21.956301,29.226633,29.568504,28.710476,28.710476,1959317.0,2010,28.596518,0.022035,0.858028,NaN
DOV,2010-01-08,22.585787,30.064552,30.077959,28.971907,29.199820,2190843.0,2010,29.226633,0.028670,1.106052,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

I executed both pandas and dask frames. When I executed daskthe return was an empty table with the type of data in each column including the new one called ma_returns. When I converted dask data frane in pandas and then added the feature I think my code ran faster and the resuls was the table above with the actual records in it

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.